In [ ]:
import sys
print(sys.argv[0])

In [ ]:
import os
HERE = os.getcwd()
HERE

In [ ]:
# https://sites.google.com/nvidia.com/graspdataset

In [ ]:
#!conda install h5py
!pip install -U scikit-learn

In [ ]:
import h5py
import numpy as np
!pip3 install mesh-to-sdf

In [ ]:
DATA_DIR = "data/examples/grasps/"
COUCH_H5 = DATA_DIR + "Table_99cf659ae2fe4b87b72437fd995483b_0.009700376721042367.h5"
grasps = h5py.File(COUCH_H5, 'r')

In [ ]:
grasps.visititems(print)

In [ ]:
grasps['object/file'][()]  # file name in shapenet?

# Get all meshes from ShapeNet

Waiting for approval, following steps will use example meshes from acronym repo

# Acronym repo: render example meshes

In [ ]:
# clone acronym, install dependencies

!git clone https://github.com/NVlabs/acronym.git

In [ ]:
%cd {HERE}/acronym

In [ ]:
!pip install -r requirements.txt
!pip install -e .
!pip install "pyglet<2"  # conflict

In [ ]:
!cat requirements.txt

## run example commands

In [ ]:
!acronym_visualize_grasps.py --mesh_root data/examples data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5


In [ ]:
!acronym_generate_scene.py --mesh_root data/examples/ --objects data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5 data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5 data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5 data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5 --support data/examples/grasps/Table_99cf659ae2fe4b87b72437fd995483b_0.009700376721042367.h5


# Render custom mesh

using examples, but in custom directory

In [ ]:
%cd {HERE}

In [ ]:
!ls

In [ ]:
MESH_ROOT = "acronym/data/examples"
GRASP_PATH = "acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5"

In [ ]:
from acronym.acronym_tools.acronym import load_mesh
obj_mesh = load_mesh(GRASP_PATH, mesh_root_dir=MESH_ROOT)
obj_mesh

In [ ]:
import trimesh
#!conda install -c conda-forge libstdcxx-ng
trimesh.Scene([obj_mesh]).show()

In [ ]:
from mesh_to_sdf import sample_sdf_near_surface
import trimesh
import pyrender
import numpy as np
points, sdf = sample_sdf_near_surface(obj_mesh, number_of_points=250000)
points, sdf = points / 2.0, sdf / 2.0
colors = np.zeros(points.shape)
colors[sdf < 0, 2] = 1
colors[sdf > 0, 0] = 1
cloud = pyrender.Mesh.from_points(points, colors=colors)
scene = pyrender.Scene()
scene.add(cloud)
viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

# Rescale custom mesh

first stage of preprocessing, to NOCS cube

In [ ]:
obj_mesh.bounds

In [ ]:
# define transformation

p1, p2 = obj_mesh.bounds

# center
c = (p1 + p2) / 2

# translation
t = - c

# scaling
s = np.min(2 / np.abs(p2 - p1))
S = s * np.eye(3)

# homogeneous transformation matrix
transform = np.eye(4, dtype=float)
transform[:3, :3] = S
transform[:3, 3] = s * t

mesh = copy(obj_mesh)
mesh.apply_transform(transform)

transform

In [ ]:
from copy import deepcopy, copy
mesh = copy(obj_mesh)
mesh.apply_transform(transform)
mesh.bounds

# sample points from mesh

second stage of preprocessing, sample points near surface

1. sample 235,000 points on the surface (signed distance 0)
2. for each point p i, we sample two Gaussian distributions with mean 0 and standard deviation 0.005 and 0.0005
3. grid points with resolution 128x128x128

store all points with grund truth(gt) signed distance(sd) values (they are known beacuse we know the mesh)

NOTE: two possibilities `trimesh.sample.sample_surface` `trimesh.sample.sample_surface_even`

In [ ]:
def sample_surface(mesh, n):
    sampled_points, _ = sample_sdf_near_surface(mesh, n)
    return np.array(sampled_points)

def sample_near(points):
    sampled_points = []
    for p in points:
        p1 = np.random.multivariate_normal(p, 0.005 * np.eye(3), 1)[0]
        p2 = np.random.multivariate_normal(p, 0.0005 * np.eye(3), 1)[0]
        sampled_points.extend([p1, p2])
    return np.array(sampled_points)

def sample_grid(res=128):
    linsp = np.linspace(-1, 1, res)
    # 128**3 = 2097152
    X, Y, Z = np.meshgrid(linsp, linsp, linsp)
    sampled_points = list(zip(X.ravel(), Y.ravel(), Z.ravel()))
    return np.array(sampled_points)

def signed_distance(mesh, query_points):
    sds = trimesh.proximity.signed_distance(mesh, query_points)
    sds = np.array(sds)  # from TrackedArray to np.array
    sds = np.expand_dims(sds, 1)  # unsqueeze
    return sds

In [ ]:
!

In [ ]:
import pandas as pd

def save_point_cloud_sdf(mesh, query_points, path):
    # get signed distance
    sds = signed_distance(mesh, query_points)

    # merge query points and sd values
    q_sd = np.hstack([query_points, sds])
    
    # save to csv
    df = pd.DataFrame(q_sd)
    df = df.round(6)  # round floats
    df.to_csv(path, sep=',', header=False, index=False)

def preprocess(mesh, save_dir='.', n=235000, res=128):
    # sample points
    p_surf = sample_surface(mesh, n)
    p_near = sample_near(p_surf)
    p_grid = sample_grid(res)
    
    save_point_cloud_sdf(mesh, p_surf, path=f'{save_dir}/surf.csv')
    save_point_cloud_sdf(mesh, p_near, path=f'{save_dir}/near.csv')
    save_point_cloud_sdf(mesh, p_grid, path=f'{save_dir}/grid.csv')

In [ ]:
preprocess(mesh, n=235000, res=128)